In [9]:
import numpy as np
import os
import glob
import math
import nibabel as nib
import cv2

# from pycimg import CImg
from skimage import measure
from skimage import morphology
from scipy import ndimage
from scipy.spatial import ConvexHull


## Nii.gz to .jpg converter

In [10]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os

def nii_to_jpg(nii_file: str, output_dir: str) -> None:
    img = nib.load(nii_file)
    img_data = img.get_fdata()
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over each slice in the third dimension (axial slices)
    for i in range(img_data.shape[2]):
        # Get the i-th slice
        slice_data = img_data[:, :, i]
        slice_normalized = ((slice_data - np.min(slice_data)) / (np.max(slice_data) - np.min(slice_data)) * 255).astype(np.uint8)
        output_file = os.path.join(output_dir, f"slice_{i:03d}.jpg")
        plt.imsave(output_file, slice_normalized, cmap='gray')

In [ ]:
for path in ['Images/IMG_0001.nii.gz', 'Images/IMG_0007.nii.gz', 'Images/IMG_0010.nii.gz', 'Images/IMG_0011.nii.gz', 'Images/IMG_0012.nii.gz', 'Images/IMG_0013.nii.gz', 'Images/IMG_0014.nii.gz']:
    output_dir = path.split('/')[-1].split('.')[0]
    nii_to_jpg(path, f"input/{output_dir}")

In [12]:
for path in ['ReferenceSegmentations/LUNGS_IMG_0001.nii.gz', 'ReferenceSegmentations/LUNGS_IMG_0007.nii.gz', 'ReferenceSegmentations/LUNGS_IMG_0010.nii.gz', 'ReferenceSegmentations/LUNGS_IMG_0011.nii.gz', 'ReferenceSegmentations/LUNGS_IMG_0012.nii.gz', 'ReferenceSegmentations/LUNGS_IMG_0013.nii.gz', 'ReferenceSegmentations/LUNGS_IMG_0014.nii.gz']:
    output_dir = path.split('/')[-1].split('.')[0]
    nii_to_jpg(path, f"output/{output_dir}")


/tmp/ipykernel_7635/4277497730.py:17: RuntimeWarning: invalid value encountered in divide
  slice_normalized = ((slice_data - np.min(slice_data)) / (np.max(slice_data) - np.min(slice_data)) * 255).astype(np.uint8)
/tmp/ipykernel_7635/4277497730.py:17: RuntimeWarning: invalid value encountered in cast
  slice_normalized = ((slice_data - np.min(slice_data)) / (np.max(slice_data) - np.min(slice_data)) * 255).astype(np.uint8)


## Segmentation

In [ ]:
fname = './Kody_Pomiary/SEG_IMG_0001.nii.gz'

foo = nib.load(fname).get_fdata()

print(foo.shape)
CImg(foo).display()

In [ ]:
labels,_ = ndimage.label(foo)

if np.sum(labels) > 0:
    props = measure.regionprops(labels)
    ids,counts = np.unique(labels,return_counts=True)
    
    for i,c,regionProp in zip(ids[1:],counts[1:],props):   #bez tła

        voxels = np.where(labels==i)
        voxels = np.asarray(voxels)
        voxels = np.swapaxes(voxels,0,1)
        chull = ConvexHull(voxels,qhull_options = 'QJ')

        dum = np.zeros(labels.shape,dtype=np.float32)
        dum[labels==i] = 1
        verts, faces, _, _ = measure.marching_cubes(dum, 0.0)
        surf = measure.mesh_surface_area(verts, faces)

        print(fname,i, \
                surf, \
                math.pow(3*c/4/math.pi,1/3)/math.pow(surf/4/math.pi,1/2), \
                regionProp.area, \
                regionProp.inertia_tensor_eigvals[0], \
                regionProp.inertia_tensor_eigvals[1], \
                regionProp.inertia_tensor_eigvals[2], \
                regionProp.major_axis_length, regionProp.minor_axis_length, \
                regionProp.bbox_area, \
                regionProp.equivalent_diameter, \
                chull.volume,chull.area)